<a href="https://colab.research.google.com/github/RDGopal/IB9LQ0-GenAI/blob/main/8_1_SQL_based_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL-based Retrieval Augemented Generation
In this tutorial we will be query a relational database to answer user queries. The contribution here is that rather than a user having to specify the query as a SQL-string (which also means they will need to understand the structure of the database), they can express their query as natural language. E.g.

_"How many students of MSBA gave 4 stars or higher on the module review?"_

Might be converted to:

```
SELECT COUNT(DISTINCT student_id)
FROM reviews
WHERE course = 'MSBA' AND AVG(review_score) >= 4;
```
We can start by setting up the instance.

You will need to use a Hugging Face token to make this work. Follow these steps:
1. Got to https://huggingface.co/
2. Click "Sign Up" in the top right corner.
3. Do the usual account sign up steps.
4. Make sure you go to your email and click on the link to confirm your account.
5. Once logged-in, click on your icon in the top right corner and select "Access tokens" (right at the bottom of the menu).
6. Click "+ Create new token".
7. Give your token a name and then scroll to the bottom to click "Create". You can ignore all the other options.
8. Copy your token secret ("hf_...") and save it somewhere on your machine (e.g. Word or Notepad).
9. Back in Colab, click on the key icon on the left hand side.
10. Click on "+ Add new secret".
11. Give the new secret the Name HF_TOKEN (please copy exactly this name).
12. Paste in your token secret from step 8 as the Value.
13. Make sure Notebook access is slid to the right. If done it will go blue and show a tick.
14. Read on!

In [ ]:
# Install required packages
!pip install -q faker sqlite-utils transformers accelerate

# Retrieve Hugging Face API token from Colab Secrets
import os
from google.colab import userdata

hf_api_token = userdata.get("HF_TOKEN")  # Set this secret via Runtime > Secrets

if hf_api_token:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_api_token
else:
    raise ValueError("Hugging Face API Token is missing. Please set it in Colab Secrets.")

If the above code worked you have added your token! This will be available from all Colab Notebooks (including future ones in these classes), although you need to follow option 13 (slide to the right to get the blue tick) to make it avaialble each time.

Now we will set up the model:

In [ ]:
# Load Falcon-7B-Instruct
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_api_token)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    token=hf_api_token
)

Much of the above should look familiar to you from ADA. Here we are using a fairly small open-source model (Falcon with 7bil parameters ... which would be considered huge five years ago but there you go).

As with any transformer architecture, we need to tokenise our data (in this case the user queries that will be translated to SQL). We will set the tokeniser to be the same as our main model (Falcon 7b).

Lastly we download the model, setting some parameters such as the use of "auto" to map tasks to our machine (basically to move stuff onto the GPU or CPU), the size of our tensor floats ("bfloat16"), that we need to trust the code associated with the model ("trust_remote_code=True") and that we use our HuggingFace token we loaded as a secret earlier.

Next we will build a SQL database:

In [ ]:
# Create a fake SQLite database
import sqlite3
from faker import Faker

fake = Faker()
conn = sqlite3.connect("people.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS people (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT,
    job TEXT,
    city TEXT,
    age INTEGER
)
""")

# Add 100 fake rows
for _ in range(100):
    cur.execute("INSERT INTO people (name, email, job, city, age) VALUES (?, ?, ?, ?, ?)", (
        fake.name(), fake.email(), fake.job(), fake.city(), fake.random_int(min=20, max=70)
    ))
conn.commit()

This one should be familiar to you from Data Management. We're simply making a SQLite table and populating ti with data (using Faker).

Next we will build our function to create SQL from natural language (via the LLM):

In [ ]:
# Generate SQL from natural language
def generate_sql(nl_query): # pass a natural language query to the function (nl_query)

    # prompt template
    prompt = f"""
      You are an expert SQL generator.
      Translate the user's natural language question into an SQLite query.
      Important! Return only the SQL code and nothing else.

      Schema:
      people(id, name, email, job, city, age)

      User question: {nl_query}
      Important! Return only the SQL code and nothing else.
      SQL query:
    """

    # tokenise the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # generate outputs
    # **inputs means upack all the inputs (tokens)
    # max_new_tokens is the maximum number of tokens to be generated (150)
    outputs = model.generate(**inputs, max_new_tokens=150)

    # decode from tokens back into language (SQL).
    # ignore special tokens such as "BOS" and "SEP"
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # extract the last part of the response (ignore the prompt, etc.)
    # this assumes that the LLM writes the prompt directly after
    # "SQL query:"
    sql = generated.split("SQL query:")[-1].strip().split("\n")[0]

    # quite possible the function won't work because the LLM doesn't return the right info
    # you can mess around to get the output you need e.g.

    sql = generated.split("SQL query:")[-1].strip().split("\n")[1].replace("`", "").replace("`", "").replace("  ","")

    # however, probably just better to use a better model


    return sql

Let's break down the "sql=" bit:

```
sql = generated.split("SQL query:")[-1].strip().split("\n")[0]
```

Firstly we are splitting the generated output effectively breaking the output into two. The first part is anything before the "SQL query:" (i.e. the end of the prompt. The second part is everything after the prompt. The _[-1]_ part means we keep just the second part (drop the prompt).

The next part ... _strip()_ ... means we get rid of any trailing spaces. E.g. "   Hello, World!   " would become "Hello, World!".

_.split("\n")_, the next part of the code, splits the remaining outputs into separate chunks for each line break ("\n") in the text. This _should_ mean we get the output SQL code as the first item, and probably some further text generated by the LLM as separate items after that. We finally use _[0]_ to return just the SQL code.

Of course, this assumes it all works properly but with a smaller model (7bil paramaters) this is not guaranteed. I had to mess around a bit taken different items and replacing different characters such as " ` " (back tick) from the output. Your results may vary and you can try to fix them (based on the SQL generated output), or otherwise do not worry. With a bigger model this shouldn't happen.

Now that we have a prompt template for the LLM, we need a function to wrap around it to process the natural language query and feed the SQL output into the database:

In [ ]:
# Query executor
def run_query(nl_query):

    # call the generate_sql function with the user query (natural language)
    sql = generate_sql(nl_query)
    # print the output
    print("SQL Generated:", sql)

    # use try in case the SQL is bad
    try:
        # run the query on the database
        result = cur.execute(sql).fetchall()

        # if it works print the first 5 results
        if result:
            print("\n Top 5 Results:")
            for row in result[:5]:
                print(row)
        else:
            print("No results found.")

    # print database error if the try fails
    except Exception as e:
        print("SQL Error:", e)

Now with our functions generated, we can test if it works!

In [ ]:
# Test the full pipeline
run_query("List the names and jobs of people over 50")

Have a play around and see if you can write more complex queries. How far can you push the 7B model?